In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
sys.path.append("../website")

In [12]:
import re
import stanza
from collections import Counter
import itertools
from title_maker_pro import datasets
import pickle
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
from words import WordIndex, Word
# stanza.download('en')  

In [ ]:
def print_words(words, f):
    for word in words:
        word_str = [word.word]
        if word.pos:
            word_str.append(f"/{word.pos}/")
        if word.topic:
            word_str.append(f"[{word.topic}]")
        print(" ".join(word_str), file=f)
        print(f"\t{word.definition}{' |n| ' if word.example is None else ''}", file=f)
        if word.example:
            print(f"\t\"{word.example}\"", file=f)

        print("", file=f)

In [4]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = datasets.Blacklist.load("/mnt/evo/projects/title-maker-pro/models/blacklist2.pickle")
model = AutoModelWithLMHead.from_pretrained("/mnt/evo/projects/title-maker-pro/models/en_dictionary_parsed_lr_00001_creativity/checkpoint-120000/").to("cuda:0")

2020-05-14 01:30:27 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-05-14 01:30:27 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-05-14 01:30:27 INFO: Use device: gpu
2020-05-14 01:30:27 INFO: Loading: tokenize
2020-05-14 01:30:29 INFO: Loading: pos
2020-05-14 01:30:29 INFO: Done loading processors!


In [ ]:
%timeit datasets.ParsedDictionaryDefinitionDataset.evaluate_creativity(tokenizer, model, blacklist, 100, 50, max_length=512)

In [7]:
def no_weird(w):
    return (
            w.word[-1] != "-"
            and "<|" not in w.definition
            and "<|" not in w.example
            and (not w.pos or "<|" not in w.pos)
            and len(w.word.split()) <= 3
            and len(w.definition.split()) >= 3
            and len(w.example.split()) >= 3
        )
         
def go(**kwargs):
    return datasets.ParsedDictionaryDefinitionDataset.generate_words(
        tokenizer, model,
        num=100000,
        max_iterations=50000, 
        blacklist=blacklist, 
        example_match_pos_pipeline=nlp,
        generation_args=dict(
            top_k=200,
            num_return_sequences=500,
            max_length=250,
            do_sample=True,
        ),
        filter_proper_nouns=True,
        user_filter=no_weird,
        dedupe_titles=True,
        min_definition_words=3,
        **kwargs
    )

# words, stats = go()
# print(stats)
# print()
# print_words(words, sys.stdout)

In [8]:
words, stats = go(use_custom_generate=True)
print(stats)
#print_words(words, sys.stdout)

2020-05-14 01:33:46 WARNING: No UPOS mapping for contraction -  unweren't  in ' I bow as if to say, it was a pleasure to stand in the world of unweren't '': NOUN
2020-05-14 01:34:27 WARNING: No UPOS mapping for adverb adjective -  nigredo  in ' Nigredo had a twin sister and was a little brother '': NOUN
2020-05-14 01:34:32 WARNING: No UPOS mapping for suffix -  -idian  in ' chalk-idian '': NOUN
2020-05-14 01:34:49 WARNING: No UPOS mapping for adverb adjective -  cutside  in ' slices of organic pasta cutside out from the food '': NOUN
2020-05-14 01:35:23 WARNING: No UPOS mapping for suffix -  -dize  in ' haze-dize '': NOUN
2020-05-14 01:35:36 WARNING: No UPOS mapping for verb adjective -  stereolize  in ' stereolized paper '': VERB
2020-05-14 01:35:38 WARNING: No UPOS mapping for colloquialize -  wahps  in ' he died wahps '': NOUN
2020-05-14 01:35:44 WARNING: No UPOS mapping for palladine <|pos|> noun -  meop  in ' we used a hot chocolate meop '': PUNCT
2020-05-14 01:36:04 WARNING: No U

iterations=8630 time=16676.84440779686 stanza_time=2301.693498849869 | items_considered 1.00@4314986, failed_match 0.87@3757017, blacklist_filtered 0.00@0, seen_filtered 0.03@112959, proper_noun_filtered 0.03@119104, example_missing 0.00@203, short_definitions 0.00@12744, example_missing_title 0.04@161160, example_pos_match_failed 0.01@40057, user_filtered 0.00@11742, returned 0.02@100000


In [ ]:
print(len(words))

In [ ]:
blacklist.contains("foolage")

In [ ]:
len(blacklist.blacklist_set)

In [ ]:
print_words(words[:50], sys.stdout)

In [ ]:
import math
from transformers import activations
import transformers

def gelu_new(x):
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

activations.ACT2FN['gelu_new'] = gelu_new

model = AutoModelWithLMHead.from_pretrained("../build/forward-dictionary-model-v1").to("cpu")
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear, torch.nn.Embedding, transformers.modeling_utils.Conv1D}, dtype=torch.qint8
)

In [ ]:
a = go2()

In [ ]:
print(tokenizer.decode(a[0]))

In [ ]:
%timeit go2()

In [14]:
from words import WordIndex, Word
def clean_example(w, example):
    return re.sub(re.escape(w), w, example, flags=re.IGNORECASE)

In [16]:
from hyphen import Hyphenator
h_en = Hyphenator('en_US')

wi = WordIndex(
    [
        Word(
            word=w.word,
            definition=w.definition,
            pos=w.pos,
            topic=w.topic,
            example=clean_example(w.word, w.example),
            syllables=h_en.syllables(w.word),
            probably_exists=False,
        ) for w in words
        
    ]
)
wi.dump("../website/data/words3.json")

In [ ]:

h_en.syllables('fancccwe')
wi2 = WordIndex.load("../website/data/words.json")
wi_p = WordIndex(
    [
        Word(
            word=w.word,
            definition=w.definition,
            pos=w.pos,
            topic=w.topic,
            example=clean_example(w.word, w.example),
            syllables=h_en.syllables(w.word)
        )
        for w in wi2.words
    ]
)

In [ ]:
wi_p.dump("../website/data/words.json")